In [14]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
X_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainy.npy')
X_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testX.npy')
y_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [31]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Input ,concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers

In [32]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [55]:
CNN_model= Sequential()
CNN_model.add(Conv1D(filters=32, kernel_size=9, strides=2, activation='relu', input_shape=(n_timesteps,n_features)))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=128, kernel_size=1, strides=1, activation='relu'))
CNN_model.add(Flatten())
CNN_model.add(Dense(n_outputs, activation='relu'))
CNN_model.summary()

LSTM_model= Sequential()
LSTM_model.add(LSTM(64,input_shape=(n_timesteps,n_features),return_sequences=True))
LSTM_model.add(LSTM(64))
LSTM_model.add(Dropout(0.1))
LSTM_model.add(Dense(128, activation='relu'))
LSTM_model.add(Dense(n_outputs, activation='relu'))
LSTM_model.summary()

inputs = Input(shape=(n_timesteps,n_features))
mergedInput= concatenate([CNN_model(inputs),LSTM_model(inputs)])
out = Dense(n_outputs, activation='softmax')(mergedInput)
model3 = Model(inputs,out)
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_67 (Conv1D)          (None, 60, 32)            1760      
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 30, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_68 (Conv1D)          (None, 28, 64)            6208      
                                                                 
 dropout_27 (Dropout)        (None, 28, 64)            0         
                                                                 
 conv1d_69 (Conv1D)          (None, 26, 128)           24704     
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 13, 128)          0         
 g1D)                                                

In [58]:
validation_data=(X_validation, y_validation)
history = model3.fit(X_training, y_training, epochs=10, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)

Epoch 1/10
207/207 [==============================] - 5s 22ms/step - loss: 0.0227 - accuracy: 0.9930 - val_loss: 0.2628 - val_accuracy: 0.9645
Epoch 2/10
207/207 [==============================] - 4s 21ms/step - loss: 0.0245 - accuracy: 0.9923 - val_loss: 0.2554 - val_accuracy: 0.9650
Epoch 3/10
207/207 [==============================] - 4s 22ms/step - loss: 0.0249 - accuracy: 0.9918 - val_loss: 0.2596 - val_accuracy: 0.9633
Epoch 4/10
207/207 [==============================] - 5s 22ms/step - loss: 0.0201 - accuracy: 0.9932 - val_loss: 0.2582 - val_accuracy: 0.9631
Epoch 5/10
207/207 [==============================] - 4s 22ms/step - loss: 0.0231 - accuracy: 0.9935 - val_loss: 0.2687 - val_accuracy: 0.9641
Epoch 6/10
207/207 [==============================] - 4s 21ms/step - loss: 0.0238 - accuracy: 0.9929 - val_loss: 0.2494 - val_accuracy: 0.9650
Epoch 7/10
207/207 [==============================] - 4s 21ms/step - loss: 0.0241 - accuracy: 0.9924 - val_loss: 0.2498 - val_accuracy: 0.9634

In [59]:
loss, accuracy = model3.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model3.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9656
Testing Accuracy: 0.9099
